# LeNet - CNN Tensorflow 예제

In [ ]:
# -*- coding: utf-8 -*-

# 1. 패키지 임포트
import tensorflow as tf
import random
from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)  # reproducibility

# 2. mnist 데이터 다운로드
mnist = input_data.read_data_sets("./temp/MNIST_data/", one_hot=True)

In [ ]:
# 3. 하이퍼파라미터 세팅
learning_rate = 0.01
epochs = 20
batch_size = 128
display_step = 1

# 4. 플레이스홀더 정의
X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])   # img 28x28x1 (black/white)
Y = tf.placeholder(tf.float32, [None, 10])

In [ ]:
# 5. 3 레이어 CNN 모델 함수
def CNN_fuction(x):

    # 6. CNN 모델의 Conv 레이어 1
    W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
    L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
    L1 = tf.nn.relu(L1)
    L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')

    # 7. CNN 모델의 Conv 레이어 2
    W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
    L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
    L2 = tf.nn.relu(L2)
    L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    L2_flat = tf.reshape(L2, [-1, 7 * 7 * 64])

    # 8. CNN 모델의 FC 레이어
    W3 = tf.get_variable("W3", shape=[7 * 7 * 64, 10], initializer=tf.contrib.layers.xavier_initializer())
    b = tf.Variable(tf.random_normal([10]))
    logits = tf.matmul(L2_flat, W3) + b
  
    return logits  


In [ ]:
# 9. CNN 모델 함수 호출
logits = CNN_fuction(X)

# 10. 손실함수와 최적화 함수
cost =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# 11. 세션 시작
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 12. 훈련
print("[CNN Trainig Start]")
print(mnist.train.num_examples / batch_size)

In [ ]:
for epoch in range(epochs):
    average_loss = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        average_loss += c / total_batch
		
	# 13. 훈련 상태(현재 에포크의 손실 값) 출력
    if epoch % display_step == 0:
        print("[Epoch] %d, [Loss] %f" % ((epoch+1), average_loss))
	#print("[Epoch] %d, [Loss] %f" % ((epoch+1), average_loss))

print("[CNN Trainig End]")


In [ ]:
# 14. 랜덤 데이터를 이용한 완성된 모델 예측
r = random.randint(0, mnist.test.num_examples - 1)
print("[Label] ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("[Prediction] : ", sess.run(
    tf.argmax(logits, 1), feed_dict={X: mnist.test.images[r:r + 1]}))